In [5]:
import datetime
from datetime import datetime as dt
import time
import requests
import pandas as pd
import numpy as np
from ta.trend import PSARIndicator

def unix_to_local_time(unix_timestamp):
    return str(dt.fromtimestamp(unix_timestamp).strftime('%Y-%m-%d %H:%M:%S'))

def calculate_heikin_ashi(df):
    ha_df = pd.DataFrame(index=df.index, columns=['HA-Open', 'HA-High', 'HA-Low', 'HA-Close'])
    ha_df['HA-Close'] = (df['open'] + df['high'] + df['low'] + df['close']) / 4
    ha_df.at[0, 'HA-Open'] = (df['open'].iloc[0] + df['close'].iloc[0]) / 2
    for i in range(1, len(df)):
        ha_df.at[i, 'HA-Open'] = (ha_df.at[i - 1, 'HA-Open'] + ha_df.at[i - 1, 'HA-Close']) / 2
        ha_df.at[i, 'HA-High'] = max(df['high'].iloc[i], ha_df.at[i, 'HA-Open'], ha_df.at[i, 'HA-Close'])
        ha_df.at[i, 'HA-Low'] = min(df['low'].iloc[i], ha_df.at[i, 'HA-Open'], ha_df.at[i, 'HA-Close'])
    result_df = pd.concat([df, ha_df], axis=1)
    return result_df

def is_last_candle_of_day(current_timestamp, timeframe_minutes):
    market_close_time = datetime.time(15, 30)
    current_time = dt.fromtimestamp(current_timestamp).time()
    next_candle_time = (datetime.datetime.combine(datetime.date.today(), current_time) + datetime.timedelta(minutes=timeframe_minutes)).time()
    return next_candle_time >= market_close_time

def trade_option(symbolID, days_before, timeframe):
    end_time_in_millis = int(time.time() * 1000)
    end_time = datetime.datetime.fromtimestamp(end_time_in_millis / 1000)

    # Calculate the start time based on the number of days before the end time
    start_time = end_time - datetime.timedelta(days=days_before)
    start_time = start_time.replace(hour=9, minute=30, second=0, microsecond=0)

    # Convert start and end times to Unix timestamps in milliseconds
    start_time_in_millis = int(start_time.timestamp() * 1000)
    end_time_in_millis = int(end_time.timestamp() * 1000)

    ourl = 'https://groww.in/v1/api/stocks_fo_data/v4/charting_service/chart/exchange/NSE/segment/FNO/'+symbolID
    url = 'https://groww.in/v1/api/charting_service/v4/chart/exchange/NSE/segment/CASH/'+'NIFTY'
    params = {
        'endTimeInMillis': str(end_time_in_millis),
        'intervalInMinutes': timeframe,
        'startTimeInMillis': str(start_time_in_millis),
    }
    
    response = requests.get(ourl, params=params)
    
    if response.status_code == 200:
        json_data = response.json()
        df = pd.DataFrame(json_data)
    
    df.drop(['changeValue', 'changePerc', 'closingPrice', 'startTimeEpochInMillis'], axis=1, inplace=True)
    ohlc = df['candles'].apply(pd.Series)
    
    ohlc.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    ohlc.drop(['volume'], axis=1, inplace=True)
    ohlc = calculate_heikin_ashi(ohlc)
    ohlc.drop(['high', 'low', 'HA-High','HA-Low'], axis=1, inplace=True)
    return ohlc

def is_first_candle_of_day(current_timestamp, timeframe_minutes):
    market_open_time = datetime.time(9, 15) 
    today_date = datetime.date.today()
    market_open_datetime = datetime.datetime.combine(today_date, market_open_time)    
    first_candle_end_time = market_open_datetime + datetime.timedelta(minutes=timeframe_minutes)
    first_candle_start_time = market_open_datetime
    current_datetime = dt.fromtimestamp(current_timestamp)    
    return first_candle_start_time <= current_datetime < first_candle_end_time

In [8]:
symbol1 = 'BANKNIFTY2460550600CE'
symbol2 = 'NIFTY2460623150CE'
days_before =0
timeframe = 15
ohlc = trade_option(symbol1, days_before, timeframe)
position = False
entry_price = None
total_profit_loss = 0
trades_taken = 0
stop_loss = 5
take_profit = 5
daily_pl =0
daily_pl_limit = 20

for i, row in ohlc.iterrows():
    if i == 0:
        continue

    if is_first_candle_of_day(row['timestamp'],timeframe):
        daily_pl =0

    if ohlc.iloc[i-1]['close'] > ohlc.iloc[i-1]['open'] and daily_pl < daily_pl_limit and daily_pl > -daily_pl_limit:
        if not position:
            position = True
            entry_price = row['open']
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if position:
        if ohlc.iloc[i-1]['close'] < ohlc.iloc[i-1]['open'] or row['open'] <= (entry_price-stop_loss) or row['open'] >= (entry_price+take_profit) or is_last_candle_of_day(row['timestamp'], timeframe):
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            daily_pl += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None



print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)}")
print(f"\nTake Away: {(round(total_profit_loss,1)*15)-(trades_taken*60)}")

Long: Entry Time: 2024-06-04 09:45:00, Entry Price: 466.05
Exit Long: Time: 2024-06-04 10:00:00, Exit Price: 493.5, P/L: 27.44999999999999

Trades count: 1 cost: 60

Total Profit/Loss: 27.4

Take Away: 351.0


In [144]:
symbol = 'NIFTY'
days_before = 1
timeframe = 10
ohlc = trade_option(symbol, days_before, timeframe)

position = False
entry_price = None
position_type = None
total_profit_loss = 0
trades_taken = 0

# Assume 'ohlc' is your OHLC data DataFrame
ohlc['HA_Close'] = (ohlc['open'] + ohlc['high'] + ohlc['low'] + ohlc['close']) / 4
ha_open = ohlc['HA_Close'].shift(1)
ohlc['HA_Open'] = ha_open.values[0]
ohlc.loc[1:, 'HA_Open'] = ha_open.values[1:]
ohlc['HA_High'] = ohlc[['HA_Open', 'HA_Close', 'high']].max(axis=1)
ohlc['HA_Low'] = ohlc[['HA_Open', 'HA_Close', 'low']].min(axis=1)

for index, row in ohlc.iterrows():
    if index == 0:
        i = 1
    else:
        i = index
        
    if row['HA_Close'] >= row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] >= ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_Low']):
        if position and position_type == 'short':
            position = False
            profit_loss = entry_price - row['open']
            total_profit_loss += profit_loss
            print(f"Exit Sell: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "long"
            trades_taken += 1
            print(f"Long: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

    if row['HA_Close'] < row['HA_Open'] and (ohlc.iloc[i-1]['HA_Close'] < ohlc.iloc[i-1]['HA_Open'] or ohlc.iloc[i-1]['HA_Open'] == ohlc.iloc[i-1]['HA_High']):
        if position and position_type == 'long':
            position = False
            profit_loss = row['open'] - entry_price
            total_profit_loss += profit_loss
            print(f"Exit Long: Time: {unix_to_local_time(row['timestamp'])}, Exit Price: {row['open']}, P/L: {profit_loss}")
            entry_price = None

        if not position:
            position = True
            entry_price = row['open']
            position_type = "short"
            trades_taken += 1
            print(f"Sell: Entry Time: {unix_to_local_time(row['timestamp'])}, Entry Price: {entry_price}")

# Print open trades
# if open_trade:
#     print("\nOpen Trades:")
#     print(f"Type: {open_trade['type']}, Entry Time: {unix_to_local_time(open_trade['entry_time'])}, Entry Price: {open_trade['entry_price']}")

print(f"\nTrades count: {trades_taken} cost: {trades_taken*60}")
print(f"\nTotal Profit/Loss: {round(total_profit_loss,1)} Total Options: {round(total_profit_loss*0.45,1)}")
if(symbol == "NIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*25)-(trades_taken*60)}")
if(symbol == "BANKNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*15)-(trades_taken*60)}")
if(symbol == "FINNIFTY"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*40)-(trades_taken*60)}")
if(symbol == "NIFTYMIDSELECT"):
    print(f"\nTake Away: {(round(total_profit_loss*0.45,1)*75)-(trades_taken*60)}")

UnboundLocalError: local variable 'df' referenced before assignment